In [1]:
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle
import random
from sklearn import svm
import sklearn


In [2]:
class TextToTensor():

    def __init__(self, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len

    def string_to_tensor(self, string_list: list) -> list:
        """
        A method to convert a string list to a tensor for a deep learning model
        """    
        string_list = self.tokenizer.texts_to_sequences(string_list)
        string_list = pad_sequences(string_list, maxlen=self.max_len)
        print('string_list',string_list)
        return string_list

In [3]:
# load data
X_train_pos=[]
Y_train_pos=[]
print('get data')
f_pos=open('train_pos_full_lem.txt',encoding ="latin-1")
for line in f_pos:
    X_train_pos.append(line.strip())
    Y_train_pos.append(1)

X_train_neg=[]
Y_train_neg=[]
f_neg=open('train_neg_full_lem.txt',encoding ="latin-1")
for line in f_neg:
    X_train_neg.append(line.strip())
    Y_train_neg.append(0)


X_train=X_train_pos+X_train_neg
Y_train=Y_train_pos+Y_train_neg
Y_train=Y_train

    

# shuffle the data : 
print('I AM HERE ')
import random
random.seed(10)

data = list(zip(X_train, Y_train))

random.shuffle(data)

X_train, labels = zip(*data)
lables=list(labels)

get data
I AM HERE 


In [4]:
with open('../data/preprocessed/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
features=[]
label=[]
i=0

embedded= np.load('../data/preprocessed/embeddings.npy')

# Get our features and our labels 
with open('../data/preprocessed/train_pos_lem.txt',encoding ="latin-1") as f:
    for line in f:
        tokens = [vocab.get(t, -1) for t in line.strip().split()]
        tokens = [t for t in tokens if t >= 0]
        if len(tokens)==0:
            features.append(np.zeros(20))
            print(i)
        else:
            features.append(np.mean(embedded[tokens,:],0))
        i+=1
        label.append(1)
        
with open('../data/preprocessed/train_neg_lem.txt') as f:
    for line in f:
        tokens = [vocab.get(t, -1) for t in line.strip().split()]
        tokens = [t for t in tokens if t >= 0]
        if len(tokens)==0:
            features.append(np.zeros(20))
            
        else:
            features.append(np.mean(embedded[tokens,:],0))
        i+=1
        label.append(0)
num_pos=i


features=np.array(features)
label=np.array(label)

# Shuffle the data 
num_row = len(label)
indices = np.random.permutation(num_row)

features_shuffle=features[indices,:]
label_shuffle=label[indices]

1924
2312
6119
7861
7862
7863
7864
7865
7866
7867
7868
7869
7870
14945
20124
21517
22869
23666
26355
27432
27764
29049
40490
43334
44426
46018
46277
46370
47455
47796
50416
51408
52444
57849
60847
63516
65338
66529
69795
70997
73576
75637
75638
75639
76402
78015
78673
78957
81907
87058
95292
96834
99314
99478


In [5]:
"""
# Preprocess the data 
with open('tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
print('create token')
tokens=tokenizer.texts_to_sequences(X_train)

print('load embedding matrix')
emb = np.load('../data/not_preprocessed/embeddings.npy')
features=[]
for i in range(len(X_train)):
    if len(tokens[i])==0 : #np.isnan(np.mean(emb[tokens[i],:])):
        features.append(np.zeros(200))
        print(tokens[i])
    else:
        features.append(np.mean(emb[tokens[i],:],0))
"""



"\n# Preprocess the data \nwith open('tokenizer.pickle', 'rb') as f:\n    tokenizer = pickle.load(f)\nprint('create token')\ntokens=tokenizer.texts_to_sequences(X_train)\n\nprint('load embedding matrix')\nemb = np.load('../data/not_preprocessed/embeddings.npy')\nfeatures=[]\nfor i in range(len(X_train)):\n    if len(tokens[i])==0 : #np.isnan(np.mean(emb[tokens[i],:])):\n        features.append(np.zeros(200))\n        print(tokens[i])\n    else:\n        features.append(np.mean(emb[tokens[i],:],0))\n"

In [6]:
features_shuffle=np.array(features_shuffle)
label_shuffle=np.array(label_shuffle)

In [7]:
print('train our model')
clf =  svm.SVC(kernel='linear') #sklearn.linear_model.LogisticRegression()
clf.fit(features_shuffle, label_shuffle)
print(clf.score(features_shuffle, label_shuffle))

train our model
0.597855


In [8]:
# save the model to disk
filename = 'model_LogReg_lem.sav'
pickle.dump(clf, open(filename, 'wb'))